### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [282]:
import pandas as pd
import numpy as py
import json
# 1. Given the order column, count how many of each product was purchased

df = pd.read_csv('food_transactions.csv')
df

# # df['order'] = df['order'].str.split(',')

# def count(x, y):
#     return x.find(y, 1)
    
            
df['S_Burger'] = df['order'].apply(lambda x: str(x)[str(x).find('S_Burger')-1])
df['S_Fries'] = df['order'].apply(lambda x: str(x)[str(x).find('S_Fries')-1])
df['S_Drink'] = df['order'].apply(lambda x: str(x)[str(x).find('S_Drink')-1])

df

,Unnamed: 0,date,branch,order,S_Burger,S_Fries,S_Drink
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",n,6,1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,n,6
...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",n,3,6
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,e
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,e
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5


In [252]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks

def family_combos(x, y, z):
    family_combos = 0
    if x == 4 and y == 4 and z == 4:
        x-=4
        y-=4
        z-=4
        family_combos += 1
        return family_combos

df['family_combos'] = df.apply(lambda row: 1 if row['S_Burger'] == 4 and row['S_Fries'] == 4 and row['S_Drink'] == 4 else 0, axis = 1)
df


,Unnamed: 0,date,branch,order,S_Burger,S_Fries,S_Drink,family_combos
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1,4,5,0
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2,4,3,0
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2,4,2,0
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",n,6,1,0
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5,n,6,0
...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",n,3,6,0
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7,6,e,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4,6,e,0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7,6,5,0


In [284]:
# 3. Calculate the total order value.
import json

with open('price_list.json', 'r') as price_list:
 file_contents = json.load(price_list)

for key in file_contents.values():
    list(key)

df['total_value'] = (df['family_combos'] * key.get('Fam_combo')) + (df['big_combo'] * key.get('Big_combo')) + (df['snack_combo']* key.get('Snack_combo')) + (df['S_Burger'] * key.get('S_Burger'))+ (df['S_Drink'] * key.get('S_Drink')) + (df['S_Fries'] * key.get('S_Fries'))



KeyError: 'family_combos'

In [ ]:
# 4. Determine which branch made the most revenue, and how much?

df.groupby("branch").sum()["total_value"].sort_values(ascending = False)

